In [ ]:
from cdcqr.common.config import CDQ_REPO_DIR
from cdcqr.common.utils import load_df, save_df
import sys
sys.path.append('/home/user/python-libs')
import cdcqr.backtest.utils as bt_util
from importlib import reload

In [ ]:
reload(bt_util)

In [ ]:
from cdcqr.backtest.utils import initcerebro

In [ ]:
from ct.utils import *
from ct.fs import *
from ct.alfafactory import *

In [ ]:
API_KEY = '1wvlrc0JsEMIWF6enI69cWiOojq'

def glassnode(url='https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m',s=1629500000):
    res = requests.get(url,params={'a': a,'i':i,s:s,'api_key': API_KEY}) #, 's':1629500000
    df = pd.read_json(res.text, convert_dates=['t']).set_index('t') 
    return df
                       
dfgsi=glassnode('https://api.glassnode.com/v1/metrics/transactions/transfers_volume_to_exchanges_mean',a='BTC',i='10m')[::-1]
#dfgsbtc=glassnode('https://api.glassnode.com/v1/metrics/market/price_usd_close',a='BTC',i='10m',s=1629500000)[::-1]

In [ ]:
dfgsi.head()

In [ ]:
! pip install pandas -U

In [ ]:
df0 = load_df('df0')
df0.columns = ['o','c','h','l']
df0['v'] = df0['h'].astype(int)
df0 = df0.resample('15Min').last().ffill()
display(df0.head(5))

In [ ]:
2021-10-12 12:14:52,414 [CRITICAL] 8544 26736 2605667408: |notify_order|Ref: 1|OrdType: 1|OrdType: Sell|Status: 1|Status: Submitted|Size: -0.1|Price: None|Price Limit: None|TrailAmount: None|TrailPercent: None|ExecType: 0|ExecType: Market|CommInfo: None|End of Session: 736129.9999999999|Info: AutoOrderedDict()|Broker: None|Alive: True|position=-0.1|brokerpos=-0.1|c=690.405|o=690.315|order=Ref: 1|OrdType: 1|OrdType: Sell|Status: 1|Status: Submitted|Size: -0.1|Price: None|Price Limit: None|TrailAmount: None|TrailPercent: None|ExecType: 0|ExecType: Market|CommInfo: None|End of Session: 736129.9999999999|Info: AutoOrderedDict()|Broker: None|Alive: True 


In [ ]:
BACKTEST=True
PRINTCANDLE=True
class Strategy(bt.Strategy):
    params = (('betsize',0.1),('lags', 20),('delay',4),('trailpercent', 3),('inflowlimit',5),('inflowlimit2',12),('nstd',2))# ('entryz', 1.2), ('exitz',1.2),('methodma','ewm'),('methodstd','ewm'),('trailpercent', 2),('liqlimit',35))
    
    def notify_order(self, order): 
        if not order.alive():
            self.order = None  # No pending orders
        logging.critical(f"|notify_order|{order}|position={self.position.size}|brokerpos={self.broker.getposition(self.datas[0]).size}|c={self.data0.close[0]}|o={self.data0.open[0]}")
         #The logic in the strategy has to remember when a position was entered, probably in notify_order by checking the date from the notified order
        #order.executed.dt which can be translated to a datetime instance like this for example: order.data.num2date(order.executed.dt)`
        #And then checking when the data hits a month later self.data.datetime, where you can get instances with self.data.datetime.datetime() for example.
    def __init__(self):
        print("strat init()")
        self.order = None
        #self.lastliq=DummyIndicator()
         
        
    def start(self):
        print("strat start()")
        self.order = None  # Avoid operrations on pending order

    def next(self):
        if not BACKTEST:
            if self.live_data==False:
                return
        if self.order:
            return  # pending order execution

        curdt=bt.num2date(self.data0.datetime[0])
        o,h,l,c,v=self.data0.open[0],self.data0.high[0],self.data0.low[0],self.data0.close[0],self.data0.volume[0]
        if PRINTCANDLE:print(f"curdt={curdt}")
        #dfpn=dfcq['btc-all_exchange-exchange-flows-netflow']['netflow_total']#.loc[:curdt]#dfcq['']#qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
        
        
        #dfpn=dfcq['btc-all_exchange-exchange-flows-inflow']['inflow_mean']#.loc[:curdt]#dfcq['']#qbars(sym='BTCUSDT@binance',freq='5Min',ts='exchtm',table='pptliqs',date=[(curdt-pd.Timedelta("10M")).date(),dt.today().date()],addwhere=', side=`sell , vol>0.01',verbose=0)
        dfpn=dfgsi#[::-1]
        dfp=dfpn[dfpn.index<curdt].head(self.p.lags)['v']
        #ipdb.set_trace()
        
        #display(dfp)
        #display(dfn)

#        df5m=df5m.resample("10Min").last()#10Min?! 5Min?
        if PRINTCANDLE:print(f"\n {dt.now()} next() self.dt={self.datetime.datetime(0)} data0close={self.data0.close[0]} data1close={self.data1.close[0] if hasattr(self,'data1') else 'not present'}  data1close={self.data2.close[0] if hasattr(self,'data2') else 'not present'}  \n")    
        if PRINTCANDLE:print(f"data0 get5={self.data0.close.get(size=5) if hasattr(self,'data0') else 'not present'}")

        if self.position.size==0: # not in the market
#             if len(dfp) and dfp.iloc[0]>dfp.mean():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
#                 logging.critical(f"dfp={dfp}")
#                 self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
#                 logging.critical(f"{self.order}")
#                 self.order = None
            #display(dfn)
            #print(f"dfn.iloc[0]<dfn.mean(){dfn.iloc[0]}<{dfn.mean()}")
            if PRINTCANDLE:print(f"dfp.iloc[0] {dfp.iloc[0]}")
            #if len(dfn) and dfn.iloc[0]<dfn.mean()-self.p.nstd*dfn.std():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
            #ipdb.set_trace()
            
            ##iloc==4 cuz 40 minutes delay
            
            if len(dfp)>5 and dfp.iloc[self.p.delay]>self.p.inflowlimit and dfp.iloc[self.p.delay]<self.p.inflowlimit2 :#dfn.mean()-self.p.nstd*dfn.std():#['netflow_total'] >df5m.iloc[-1]['netflow_total'].mean():#self.p.liqlimit :# and self.data1.reserve<10000000:
            # and  momentum is not strong   use rsi from bt?
            #display(dfn)
                #self.order = self.buy(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                self.order = self.sell(data=self.data0,exectype=bt.Order.Market,size=self.p.betsize)
                
                logging.critical(f"{self.order}")
                self.order = None

        elif self.order is None: # Position in Market
            #self.order = self.sell(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            self.order = self.buy(data=self.data0,exectype=bt.Order.StopTrail,trailpercent=self.p.trailpercent/100,price=self.data0.open[0],size=self.p.betsize,params={'reduceOnly':True,'callbackRate':self.p.trailpercent}) #If no price is specified, the latest close price is used
            
            logging.critical(f"{self.order}")
            #close_position = binance.create_order(symbol=symbol, type="MARKET", side="buy", amount=pos['positionAmt'], params={"reduceOnly": True}) 

    def notify_data(self, data, status, *args, **kwargs):
        if data._getstatusname(status) == 'LIVE':
            self.live_data = True
        else:
            self.live_data = False


    def log(self,txt,dummy=True):
        logging.debug(txt)


In [ ]:
        
def btstrat(df0,lags=20,nstd=2,trailpercent=3,inflowlimit=5,inflowlimit2=12,delay=4,methodma='ma',methodstd='ma',verbose=0,plot=False):        

    cerebro=initcerebro(multi=False,futures=True,fees=0.0006,margin=0.1)#,margin=0.1,mult=10.)fees=0.0006
    #df0=dfcq['btc-binance-market-data-price-usd'].loc['2021'].rename(columns={'price_usd_open':'o','price_usd_high':'h','price_usd_low':'l','price_usd_close':'c'})[::-1]
    data0=bt.feeds.PandasData(dataname=df0.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None,timeframe=bt.TimeFrame.Minutes,compression=1)
    cerebro.adddata(data0,name='BTCUSD@cq')
#   data1=bt.feeds.PandasData(dataname=df1.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None)#,timeframe=bt.TimeFrame.Hours)
#   cerebro.adddata(data1,name='BTCUSDT_210326@binance')

#    data0=KDBFeed(sym='ETHUSD_210326@binance',freq='1H',ts='exchtm',table=None, date=[date(2021,2,10),date(2021,3,25)],addwhere=', vol>0.01')
#    cerebro.adddata(data0,name='BTCUSD_210326@binance')
    
#    data1=KDBFeed(sym='ETHUSDT_210326@binance',freq="1H",table='ppttrades',addwhere=', vol>0.01',date=[date(2021,2,10),date(2021,3,25)])
#    cerebro.adddata(data1,name='BTCUSDT_210326@binance')

    cerebro.addstrategy(Strategy,lags=lags,nstd=nstd,trailpercent=trailpercent,inflowlimit=inflowlimit,inflowlimit2=inflowlimit2,delay=delay)#,nstdh=nstdh,nstdl=nstdl,methodma=methodma,methodstd=methodstd)
    #cerebro.addsizer(Sizer10Percent)

    backtest = cerebro.run(runonce=False,tradehistory=True)
    btres = backtest[0]
    
    rets=pd.DataFrame.from_dict(dict(btres.analyzers.timereturn.get_analysis()),orient='index', columns=['ret'])
    if plot:
        
        for k in ['total','pnl']:#backtest[0].analyzers.ta.get_analysis():
            print (k,dict(backtest[0].analyzers.ta.get_analysis()[k]))
        print(dict(backtest[0].analyzers.sharpe.get_analysis()))

        printTradeAnalysis(cerebro, backtest[0].analyzers)
        print("amount,price,sid,symbol,value")
        display(DF(btres.analyzers.tradelist.get_analysis()).set_index('datein'))
        
        rcsave=plt.rcParams['figure.figsize']
        plt.rcParams['figure.figsize']=[30, 16]
        cerebro.plot(style='line', rowsmajor=1, rowsminor=1,volume=True, iplot=False, barup='green', bardown='grey',fmt_x_ticks = '%Y-%b-%d %H:%M',fmt_x_data = '%Y-%b-%d %H:%M')
        plt.rcParams['figure.figsize']=rcsave
        
        #getinds(cerebro)[0].iplot()
    try:
        nt=btres.analyzers.ta.get_analysis()['total']['closed']
    except:
        nt=0
    try:
        sr=btres.analyzers.sharpe.get_analysis()['sharperatio']
    except:
        sr=0
    return {'cerebro':cerebro,'btres':btres,'rets':rets,'nt':nt,'SR':sr}#,'ind':bb}
df0=qbars('BTCUSDT@binance',freq="1Min",table='ppttrades',date=[date(2021,1,1),date(2021,8,10)],addwhere=', vol>0.01')
#df0['volume']=1
btstrat(df0,lags=200,inflowlimit=5,inflowlimit2=12,trailpercent=3,delay=0,plot=True)

In [ ]:
cerebro=initcerebro(multi=False,futures=True,fees=0.0006,margin=0.1)#,margin=0.1,mult=10.)fees=0.0006

In [ ]:
data0=bt.feeds.PandasData(dataname=df0.rename(columns={'o':'open','h':'high','l':'low','c':'close','v':'volume'}),openinterest=None,timeframe=bt.TimeFrame.Minutes,compression=1)

In [ ]:
cerebro.adddata(data0,name='BTCUSD@cq')

In [ ]:
lags=20
nstd=2
trailpercent=3
inflowlimit=5
inflowlimit2=12
delay=4
methodma='ma'
methodstd='ma'
verbose=0
cerebro.addstrategy(Strategy,lags=lags,nstd=nstd,trailpercent=trailpercent,inflowlimit=inflowlimit,inflowlimit2=inflowlimit2,delay=delay)#,nstdh=nstdh,nstdl=nstdl,methodma=methodma,methodstd=methodstd)
    #cerebro.addsizer(Sizer10Percent)


In [ ]:
backtest = cerebro.run(runonce=False,tradehistory=True)

In [ ]:
backtest

In [ ]:
btres = backtest[0]
    
rets=pd.DataFrame.from_dict(dict(btres.analyzers.timereturn.get_analysis()),orient='index', columns=['ret'])

In [ ]:
backtest[0].analyzers.ta.get_analysis()

In [ ]:
plot=True
if plot:

    for k in ['total']:#backtest[0].analyzers.ta.get_analysis():
        print (k,dict(backtest[0].analyzers.ta.get_analysis()[k]))
    print(dict(backtest[0].analyzers.sharpe.get_analysis()))

    printTradeAnalysis(cerebro, backtest[0].analyzers)
    print("amount,price,sid,symbol,value")
    display(DF(btres.analyzers.tradelist.get_analysis()).set_index('datein'))

    rcsave=plt.rcParams['figure.figsize']
    plt.rcParams['figure.figsize']=[30, 16]
    cerebro.plot(style='line', rowsmajor=1, rowsminor=1,volume=True, iplot=False, barup='green', bardown='grey',fmt_x_ticks = '%Y-%b-%d %H:%M',fmt_x_data = '%Y-%b-%d %H:%M')
    plt.rcParams['figure.figsize']=rcsave


In [ ]:
backtest[0].analyzers.ta.get_analysis().keys()